## Convert PyTorch model to ONNX and OpenVINO IR

In [7]:
import sys
sys.path.append("../utils")
import time
from pathlib import Path

import cv2
import numpy as np
import torch
from IPython.display import Markdown, display
from fastseg import MobileV3Large
from openvino.runtime import Core

from notebook_utils import CityScapesSegmentation, segmentation_map_to_image, viz_result_image

### Settings

In [8]:
IMAGE_WIDTH = 1024  # Suggested values: 2048, 1024 or 512. The minimum width is 512.
# Set IMAGE_HEIGHT manually for custom input sizes. Minimum height is 512
IMAGE_HEIGHT = 1024 if IMAGE_WIDTH == 2048 else 512
DIRECTORY_NAME = "model"
BASE_MODEL_NAME = DIRECTORY_NAME + f"/fastseg{IMAGE_WIDTH}"

# Paths where PyTorch, ONNX and OpenVINO IR models will be stored
model_path = Path(BASE_MODEL_NAME).with_suffix(".pth")
onnx_path = model_path.with_suffix(".onnx")
ir_path = model_path.with_suffix(".xml")

### Download the Fastseg model

In [9]:
print("Downloading the Fastseg model (if it has not been downloaded before)....")
model = MobileV3Large.from_pretrained().eval()
print("Loaded PyTorch Fastseg model")

# Save the model
model_path.parent.mkdir(exist_ok=True)
torch.save(model.state_dict(), str(model_path))
print(f"Model saved at {model_path}")

Loading pretrained model mobilev3large-lraspp with F=128...
Loaded PyTorch Fastseg model
Model saved at model\fastseg1024.pth


## ONNX model Conversion

### Convert PyTorch model to ONNX

In [10]:
if not onnx_path.exists():
    dummy_input = torch.randn(1, 3, IMAGE_HEIGHT, IMAGE_WIDTH)

    # For the Fastseg model, setting do_constant_folding to False is required
    # for PyTorch>1.5.1
    torch.onnx.export(
        model,
        dummy_input,
        onnx_path,
        opset_version=11,
        do_constant_folding=True
    )
    print(f"ONNX model exported to {onnx_path}.")
else:
    print(f"ONNX model {onnx_path} already exists.")

ONNX model model\fastseg1024.onnx already exists.


In [11]:
mo_command = f"""mo
                 --input_model "{onnx_path}"
                 --input_shape "[1, 3, {IMAGE_HEIGHT}, {IMAGE_WIDTH}]"
                 --mean_values="[123.675, 116.28, 103.53]"
                 --scale_values="[58.395, 57.12, 57.375]"
                 --data_type FP16
                 --output_dir "{model_path.parent}"
"""

mo_command = " ".join(mo_command.split())
print("Model Optimizer command to convert the ONNX model to openVINO")
display(Markdown(f"`{mo_command}`"))

Model Optimizer command to convert the ONNX model to openVINO


`mo --input_model "model\fastseg1024.onnx" --input_shape "[1, 3, 512, 1024]" --mean_values="[123.675, 116.28, 103.53]" --scale_values="[58.395, 57.12, 57.375]" --data_type FP16 --output_dir "model"`

In [12]:
if not ir_path.exists():
    print("Exporting ONNX model to IR... This may take a few minutes")
    mo_result = %sx $mo_command
    print("\n".join(mo_result))
else:
    print(f"IR model {ir_path} already exists")

Exporting ONNX model to IR... This may take a few minutes
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	c:\Users\hokut\デスクトップ\practice\LearnOpenVino\006ConvertModel\model\fastseg1024.onnx
	- Path for generated IR: 	c:\Users\hokut\デスクトップ\practice\LearnOpenVino\006ConvertModel\model
	- IR output name: 	fastseg1024
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1, 3, 512, 1024]
	- Source layout: 	Not specified
	- Target layout: 	Not specified
	- Layout: 	Not specified
	- Mean values: 	[123.675, 116.28, 103.53]
	- Scale values: 	[58.395, 57.12, 57.375]
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- User transformations: 	Not specified
	- Reverse input channels: 	False
	- Enable IR generation for fixed input shape: 	False
	- Use the transformations config file: 	None

### Show results

In [13]:
def normalize(image: np.ndarray) -> np.ndarray:
    image = image.astype(np.float32)
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    image /= 255.0
    image -= mean
    image /= std
    return image

In [ ]:
image_filename = "data/street.jpg"
image = cv2.cvtColor(cv2.imread(image_filename), cv2.COLOR_BGR2RGB)

resized_image = cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT))
normalized_image = normalize(resized_image)

# Convert the resized images to network input shape
input_image = np.expand_dims(np.transpose(resized_image, (2, 0, 1)), 0)
normalized_input_image = np.expand_dims(np.transpose(normalized_image, (2, 0, 1)), 0)

### Load the OpenVINO IR Network and Run Inference on the ONNX model

In [ ]:
ie = Core()
model_onnx = ie.read_model(model=onnx_path)
compiled_model_onnx = ie.compile_model(model=model_onnx, device_name="CPU")

output_layer_onnx = compiled_model_onnx.output(0)

res_onnx = compiled_model_onnx([normalized_input_image])[output_layer_onnx]

In [ ]:
# Convert network result to segmentation map and display the result
result_mask_onnx = np.squeeze(np.argmax(res_onnx, axis=1)).astype(np.uint8)
viz_result_image(
    image,
    segmentation_map_to_image(result_mask_onnx, CityScapesSegmentation.get_colormap()),
    resize=True
)